In [1]:
import pandas as pd
import numpy as np
import csv
import re

In [3]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin 
import requests
import re
from selenium import webdriver
import time
from selenium.webdriver.support.ui import WebDriverWait as wait

# Download IMDB's Top 250 data
url = 'http://www.imdb.com/chart/top'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

movies = soup.select('td.titleColumn')
links = [a.attrs.get('href') for a in soup.select('td.titleColumn a')]
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
ratings = [b.attrs.get('data-value') for b in soup.select('td.posterColumn span[name=ir]')]
votes = [b.attrs.get('data-value') for b in soup.select('td.ratingColumn strong')]

imdb = []
dataset = []
# Store each item into dictionary (data), then put those into a list (imdb)
for index in range(0, 1):
    # Seperate movie into: 'place', 'title', 'year'
    movie_string = movies[index].get_text()
    movie = (' '.join(movie_string.split()).replace('.', ''))
    movie_title = movie[len(str(index))+1:-7]
    year = re.search('\((.*?)\)', movie_string).group(1)
    place = movie[:len(str(index))-(len(movie))]
    data = {"movie_title": movie_title,
#            "year": year,
#             "place": place,
#             "star_cast": crew[index],
            "rating": ratings[index],
#             "vote": votes[index],
            "link": links[index]}
    imdb.append(data)

for item in imdb:
    #print(item['place'], '-', item['movie_title'], '('+item['year']+') -', 'Starring:', item['star_cast'],  item['link'])
    url =  'http://www.imdb.com'+ item['link'][:17]+'reviews'
    driver = webdriver.Chrome('/home/dev/Downloads/chromedriver_linux64/chromedriver')
    driver.get(url)

    labtn = driver.find_element_by_id('load-more-trigger')
    labtn.click()

    wait(driver, 15).until(lambda x: len(driver.find_elements_by_css_selector("div.lister-item-content")) > 39)
    source_code = driver.page_source
    soup = BeautifulSoup(source_code, 'lxml')
    containers=soup.find_all('div', class_ = 'lister-item-content')
    
    
    reviews = []
    user_ratings=[]
    for container in containers:
        if container.find('a', class_ = 'title' ) is not None:
            title = container.a.text.strip()
            
            bar = container.find('div', class_ = 'ipl-ratings-bar')
            if bar is None:
                continue
            else:
                rating = bar.text.strip()  
                content_cont = container.find('div', class_ = 'text show-more__control')
                if content_cont is not None:
                    content = content_cont.text.strip()
                    #content=' '.join(re.split(r'\t+',content))
                    #print(content)
                else:
                    content_cont = container.find('div', class_ = 'text show-more__control clickable')
                    content = content_cont.text.strip()
                content = title+' '+content
                content=' '.join(re.split(r'\t+',content))
                reviews.append(content)
                user_ratings.append(rating)
    
   
    #print(movies) 
    driver.quit()
    url1 =  'http://www.imdb.com'+ item['link'][:17]+'plotsummary'
    #print(url1)
    response = requests.get(url1)
    soup = BeautifulSoup(response.text, 'lxml')
    containers=soup.find_all('ul', {'id':'plot-synopsis-content'})
    synopsis=""
    for temp in containers:
        for temp1 in temp.find_all('li'):
            synopsis=temp1.text.strip()
            #print(synopsis)
    movies = {
        "title" : item['movie_title'],
        "rating" : item['rating'],
        "reviews": reviews,
        "user_ratings":user_ratings,
        "synopsis":synopsis
    }
    dataset.append(movies)
    

In [4]:
url = 'https://www.imdb.com/chart/bottom'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

movies = soup.select('td.titleColumn')
links = [a.attrs.get('href') for a in soup.select('td.titleColumn a')]
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
ratings = [b.attrs.get('data-value') for b in soup.select('td.posterColumn span[name=ir]')]
votes = [b.attrs.get('data-value') for b in soup.select('td.ratingColumn strong')]

imdb = []
# Store each item into dictionary (data), then put those into a list (imdb)
for index in range(0, 1):
    # Seperate movie into: 'place', 'title', 'year'
    movie_string = movies[index].get_text()
    movie = (' '.join(movie_string.split()).replace('.', ''))
    movie_title = movie[len(str(index))+1:-7]
    year = re.search('\((.*?)\)', movie_string).group(1)
    place = movie[:len(str(index))-(len(movie))]
    data = {"movie_title": movie_title,
#            "year": year,
#             "place": place,
#             "star_cast": crew[index],
            "rating": ratings[index],
#             "vote": votes[index],
            "link": links[index]}
    imdb.append(data)

for item in imdb:
    #print(item['place'], '-', item['movie_title'], '('+item['year']+') -', 'Starring:', item['star_cast'],  item['link'])
    url =  'http://www.imdb.com'+ item['link'][:17]+'reviews'
    driver = webdriver.Chrome('/home/dev/Downloads/chromedriver_linux64/chromedriver')
    driver.get(url)

    labtn = driver.find_element_by_id('load-more-trigger')
    labtn.click()

    wait(driver, 15).until(lambda x: len(driver.find_elements_by_css_selector("div.lister-item-content")) > 39)
    source_code = driver.page_source
    soup = BeautifulSoup(source_code, 'lxml')
    containers=soup.find_all('div', class_ = 'lister-item-content')
   
    reviews = []
    user_ratings=[]
    for container in containers:
        if container.find('a', class_ = 'title' ) is not None:
            title = container.a.text.strip()
            title=' '.join(re.split(r'\t+',title))
            bar = container.find('div', class_ = 'ipl-ratings-bar')
            if bar is None:
                continue
            else:
                rating = bar.text.strip()  
                content_cont = container.find('div', class_ = 'text show-more__control')
                if content_cont is not None:
                    content = content_cont.text.strip()
                    content=' '.join(re.split(r'\t+',content))
                else:
                    content_cont = container.find('div', class_ = 'text show-more__control clickable')
                    content = content_cont.text.strip()
                content = title+' '+content
                reviews.append(content)
                user_ratings.append(rating)
    
   
    #print(movies) 
    driver.quit()
    url1 =  'http://www.imdb.com'+ item['link'][:17]+'plotsummary'
    response = requests.get(url1)
    soup = BeautifulSoup(response.text, 'lxml')
    containers=soup.find_all('ul', {'id':'plot-synopsis-content'})
    synopsis=""
    for temp in containers:
        for temp1 in temp.find_all('li'):
            synopsis=temp1.text.strip()
            #print(synopsis)
    movies = {
        "title" : item['movie_title'],
        "rating" : item['rating'],
        "reviews": reviews,
        "user_ratings":user_ratings,
        "synopsis":synopsis
    }
    dataset.append(movies)
    

In [5]:
url = 'https://www.imdb.com/list/ls058813655/?sort=list_order,asc&st_dt=&mode=simple&page=1&ref_=ttls_vw_smp'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

# movies = soup.select('td.titleColumn')
# links = [a.attrs.get('href') for a in soup.select('td.titleColumn a')]
# crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
# ratings = [b.attrs.get('data-value') for b in soup.select('td.posterColumn span[name=ir]')]
# votes = [b.attrs.get('data-value') for b in soup.select('td.ratingColumn strong')]

movies = soup.find_all('div',class_='lister-col-wrapper')

imdb = []
# Store each item into dictionary (data), then put those into a list (imdb)
for index in range(0, 1):
    # Seperate movie into: 'place', 'title', 'year'
#     movie_string = movies[index].get_text()
#     movie = (' '.join(movie_string.split()).replace('.', ''))
#     movie_title = movie[len(str(index))+1:-7]
    movie_title = movies[index].a.text
    rating_temp = movies[index].find('div',class_='col-imdb-rating')
    rating = rating_temp.strong.text.strip()
    link=movies[index].a.attrs.get('href')
#     year = re.search('(.*?)(.∗?)', movie_string).group(1)
#     place = movie[:len(str(index))-(len(movie))]
    data = {"movie_title": movie_title,
#             "year": year,
#             "place": place,
#             "star_cast": crew[index],
            "rating": rating,
#             "vote": votes[index],
            "link": link}
    imdb.append(data)

for item in imdb:
    #print(item['place'], '-', item['movie_title'], '('+item['year']+') -', 'Starring:', item['star_cast'],  item['link'])
    url =  'http://www.imdb.com'+ item['link'][:17]+'reviews'
    driver = webdriver.Chrome('/home/dev/Downloads/chromedriver_linux64/chromedriver')
   # driver = webdriver.Chrome('/home/udbhav/Downloads/chromedriver')
    driver.get(url)

    labtn = driver.find_element_by_id('load-more-trigger')
    labtn.click()

    wait(driver, 15).until(lambda x: len(driver.find_elements_by_css_selector("div.lister-item-content")) > 39)
    source_code = driver.page_source
    soup = BeautifulSoup(source_code, 'lxml')
    containers=soup.find_all('div', class_ = 'lister-item-content')
    reviews = []
    user_ratings=[]
    for container in containers:
        if container.find('a', class_ = 'title' ) is not None:
            title = container.a.text.strip()
            title=' '.join(re.split(r'\t+',title))
            bar = container.find('div', class_ = 'ipl-ratings-bar')
            if bar is None:
                continue
            else:
                rating = bar.text.strip()  
                content_cont = container.find('div', class_ = 'text show-more__control')
                if content_cont is not None:
                    content = content_cont.text.strip()
                    content=' '.join(re.split(r'\t+',content))
                else:
                    content_cont = container.find('div', class_ = 'text show-more__control clickable')
                    content = content_cont.text.strip()
                content = title+' '+content
                reviews.append(content)
                user_ratings.append(rating)
    
   
    #print(movies) 
    driver.quit()
    url1 =  'http://www.imdb.com'+ item['link'][:17]+'plotsummary'
    response = requests.get(url1)
    soup = BeautifulSoup(response.text, 'lxml')
    containers=soup.find_all('ul', {'id':'plot-synopsis-content'})
    synopsis=""
    for temp in containers:
        for temp1 in temp.find_all('li'):
            synopsis=temp1.text.strip()
            #print(synopsis)
    movies = {
        "title" : item['movie_title'],
        "rating" : item['rating'],
        "reviews": reviews,
        "user_ratings":user_ratings,
        "synopsis":synopsis
    }
    dataset.append(movies)
    
df=pd.DataFrame(dataset)
#print(dataset)
# f = open("sample1.tsv", "w")
# writer = csv.DictWriter(
#     f, fieldnames=["title", "rating", "reviews","user_ratings","synopsis"])
# writer.writeheader()
# writer.writerows(dataset)
# f.close()
# df = pd.read_csv('sample1.tsv')
df.to_csv('sample1.tsv',sep='\t')
print(df)

               rating                                            reviews  \
0   9.217045300620226  [Tied for the best movie I have ever seen Why ...   
1  1.9975821404178076  [The fact that aspiring actors, directors, and...   
2                 8.5  [Unlike anything ever done in the history of c...   

                                            synopsis  \
0  In 1947, Andy Dufresne (Tim Robbins), a banker...   
1  The movie opens with spoofing "Armageddon" and...   
2  SPOILER: Thanos and his Children - Proxima Mid...   

                      title                                       user_ratings  
0  The Shawshank Redemption  [10/10, 10/10, 10/10, 10/10, 10/10, 10/10, 10/...  
1            Disaster Movie  [1/10, 1/10, 1/10, 1/10, 1/10, 1/10, 1/10, 1/1...  
2    Avengers: Infinity War  [10/10, 10/10, 10/10, 9/10, 10/10, 10/10, 10/1...  
